# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,altoona,40.5187,-78.3947,299.71,94,100,1.33,US,1721243403
1,1,lengshuitan,26.4111,111.5956,299.90,75,100,2.42,CN,1721243404
2,2,kurchum,48.5722,83.6542,292.04,71,6,1.40,KZ,1721243405
3,3,bredasdorp,-34.5322,20.0403,284.65,75,100,1.51,ZA,1721243226
4,4,yaren,-0.5472,166.9160,301.00,73,100,1.10,NR,1721243407


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
city_hmd_map = city_data_df.hvplot.points('Lng', 'Lat',
                                          geo = True,
                                          size = 'Humidity',
                                          color = 'City',
                                          tiles = 'OSM',
                                          frame_width = 700,
                                          frame_height = 500,
                                          scale = 1.5,
                                          alpha = 0.5,
                                          xlabel = 'Longitude',
                                          ylabel = 'Latitude'
                                         )

# Display the map
city_hmd_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# ideal conditions
id_min_temp = 300
id_max_temp = 310
id_wind_speed = 3
id_cloudiness = 3

# filter
ideal_cond_df = city_data_df[(city_data_df['Max Temp'] > id_min_temp) &
                            (city_data_df['Max Temp'] < id_max_temp) &
                            (city_data_df['Wind Speed'] < id_wind_speed) &
                            (city_data_df['Cloudiness'] <= id_cloudiness)]

# Drop any rows with null values
ideal_cond_df.dropna(inplace = True)

# Display sample data
ideal_cond_df

/var/folders/jv/hqpf78ss49q3n6c5hsg4mq5w0000gn/T/ipykernel_22243/1493822383.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_cond_df.dropna(inplace = True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,conceicao do araguaia,-8.2578,-49.2647,309.05,17,0,2.74,BR,1721243443
38,38,laguna,38.4210,-121.4238,303.50,44,0,2.06,US,1721243448
66,66,constantia,44.1833,28.6500,302.53,68,0,2.36,RO,1721243231
77,77,dawson creek,55.7666,-120.2362,302.95,23,0,2.57,CA,1721243492
101,101,enoch,37.7733,-113.0244,304.29,31,0,2.57,US,1721243522
166,166,campos belos,-13.0367,-46.7717,304.24,15,0,2.76,BR,1721243596
213,213,turpan,42.9333,89.1667,309.00,23,0,1.01,CN,1721243652
242,242,newman,37.3138,-121.0208,303.34,15,0,2.86,US,1721243684
357,357,cody,44.5263,-109.0565,304.26,29,0,2.57,US,1721243817
366,366,portel,-1.9356,-50.8211,306.99,35,0,1.98,BR,1721243827


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cond_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df ['Hotel Name'] = " "

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
34,conceicao do araguaia,BR,-8.2578,-49.2647,17,
38,laguna,US,38.4210,-121.4238,44,
66,constantia,RO,44.1833,28.6500,68,
77,dawson creek,CA,55.7666,-120.2362,23,
101,enoch,US,37.7733,-113.0244,31,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories':'accommodation.hotel',
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
conceicao do araguaia - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
constantia - nearest hotel: Hotel Dali
dawson creek - nearest hotel: Comfort Inn
enoch - nearest hotel: Travelodge Cedar City
campos belos - nearest hotel: No hotel found
turpan - nearest hotel: Oriental
newman - nearest hotel: No hotel found
cody - nearest hotel: Comfort Inn at Buffalo Bill Village Resort
portel - nearest hotel: No hotel found
colonia menno - nearest hotel: Hotel Mora
qingyuan - nearest hotel: 好来登国际酒店
san borja - nearest hotel: No hotel found
baharly - nearest hotel: No hotel found
campo maior - nearest hotel: Hotel Pousada do Lago
abilene - nearest hotel: DoubleTree by Hilton Abilene Downtown Convention Center
sakakah - nearest hotel: فندق النزل
immokalee - nearest hotel: Seminole Casino Hotel Immokalee
clearlake - nearest hotel: Best Western El Grande Inn
argo - nearest hotel: Apollon Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
34,conceicao do araguaia,BR,-8.2578,-49.2647,17,No hotel found
38,laguna,US,38.4210,-121.4238,44,Holiday Inn Express & Suites
66,constantia,RO,44.1833,28.6500,68,Hotel Dali
77,dawson creek,CA,55.7666,-120.2362,23,Comfort Inn
101,enoch,US,37.7733,-113.0244,31,Travelodge Cedar City
166,campos belos,BR,-13.0367,-46.7717,15,No hotel found
213,turpan,CN,42.9333,89.1667,23,Oriental
242,newman,US,37.3138,-121.0208,15,No hotel found
357,cody,US,44.5263,-109.0565,29,Comfort Inn at Buffalo Bill Village Resort
366,portel,BR,-1.9356,-50.8211,35,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points('Lng', 'Lat',
                                   geo = True,
                                   size = 'Humidity',
                                   color = 'City',
                                   tiles = 'OSM',
                                   frame_width = 700,
                                   frame_height = 500,
                                   scale = 1.5,
                                   alpha = 0.5,
                                   xlabel = 'Longitude',
                                   ylabel = 'Latitude',
                                   hover_cols = ['Hotel Name', 'Country']
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)